In [1]:
import spacy
import pickle

import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training.example import Example

This is where the NER model will be trained to recognize the missing entities

In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
ner=nlp.get_pipe("ner")

Open the gathered training data

In [7]:
file = open("../stored_data/NER_train_data.obj",'rb')
TRAIN_DATA = pickle.load(file)
file.close()

Add new label entities

In [8]:
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [9]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
import warnings
warnings.filterwarnings("ignore", message=r"\[W030\]", category=UserWarning)

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(10):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}

    count = 0

    for text, annotations in TRAIN_DATA:
        count += 1
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update(
                    [example],  # batch of texts
                    drop=0.2,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses, count / len(TRAIN_DATA),end="\r")

In [11]:
nlp.to_disk('ner_v1.0')

In [12]:
nlp.to_disk('models/ner_v1.0')

In [ ]:
import warnings
import tqdm
warnings.filterwarnings("ignore", message=r"\[W030\]", category=UserWarning)

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(10):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}

    count = 0

    for batch in spacy.util.minibatch(TRAIN_DATA, size=5):
        for text, annotations in batch:
            count += 1
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update(
                        [example],  # batch of texts
                        drop=0.2,  # dropout - make it harder to memorise data
                        losses=losses,

                    )
            print("Losses", losses, count / len(TRAIN_DATA), "iteration :", iteration,end="\r")